[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bsbodden/redis_vss_getting_started/blob/main/vector_similarity_with_redis.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/bsbodden/redis_vss_getting_started/main/assets/nbviewer-shield.svg)](https://nbviewer.org/github/bsbodden/redis_vss_getting_started/blob/main/vector_similarity_with_redis.ipynb)

# Vector Similarity Search with Redis

## The "Unstructured Data" Problem

Today, about 80% of the data organizations generate is "unstructured"; data that either does not have a well-defined schema or cannot be restructured into a familiar columnar format. Typical examples of unstructured data include free-form text, images, videos, and sound clips. Moreover, this data imbalance towards unstructured data is expected to grow in the coming decades.

Unstructured data is high-dimensional and noisy,    making it more challenging to analyze and interpret using traditional methods. But it is also packed with information and meaning.  

Traditionally, unstructured data is processed to extract specific features, effectively turning it into structured data. Once in the realm of structured data, we can search the data with SQL queries (if stored in a relational database) or with a text search engine. 
The approach of transforming unstructured data into structured data has a few issues; first, engineering features out of structured data can be computationally expensive and error-prone, significantly delaying when we can effectively use the data. And secondly, we are likely losing fidelity and information since when unstructured data is converted to structured data, some unique or 'latent' features might be lost or overlooked because they can't be easily categorized or quantified.

## Enter "Vector Databases"

An approach to dealing with unstructured data is to "vectorize" such data. By "vectorizing," we mean to somehow convert something like a text passage, an image, a video, or a song into a flat sequence of numbers representing a particular piece of data. These "vectors" are representations of the data in N-dimensional space. By vectorizing, we gain the ability to use linear algebra techniques to compare, group, and operate on our data. This is the foundation of a Vector Database, the ability to store and operate on vectors.
This approach is not new and has been around for eons. The difference today is how the techniques for generating the vectors have advanced.

## Using Machine Learning "Embeddings" as Vectors

Traditional methods for converting unstructured data into vector form include Bag-of-Words (BoW) and TF-IDF (Term Frequency-Inverse Document Frequency) for textual data. For categorical data, one-hot Encoding is a commonly used approach. And Hashing and Feature Extraction techniques, such as edge detection, texture analysis, or color histograms, have been employed for high-dimensionality data like images.

While powerful in their own right, these approaches reveal limitations when confronted with high-dimensional and intricate data forms like long text passages, images, and audio. Consider, for example, how a text passage could be restructured—through sentence rearrangement, synonym usage, or alterations in narrative style. Such Simple modifications could effectively sidestep techniques like Bag-of-Words, preventing systems using the generated encodings from identifying texts with similar meanings.

This is where advancements in Machine Learning, particularly Deep Learning, make their mark. Machine Learning models have facilitated the rise of 'embeddings' as a widely embraced method for generating dense, low-dimensional vector representations. Given a suitable model for the task at hand, the generated embeddings can encapsulate complex patterns and semantic meanings inherent in data, thus overcoming the limitations of their traditional counterparts.

### Generating Embeddings for the "Bikes" Dataset

To investigate Vector Similarity, we'll use a subset of the Bikes dataset, a relatively simple synthetic dataset. The dataset has 11 bicycle records in a JSON file named `bikes.json` and includes the fields `model`, `brand`, `price`, `type`, `specs`, and `description`. The `description` field is particularly interesting for our purposes since it consists of a free-form textual description of a bicycle.

#### Loading JSON "Bikes" Dataset

Let's load the bikes dataset as a JSON array:

In [1]:
import requests
import json

url = 'https://raw.githubusercontent.com/bsbodden/redis_vss_getting_started/main/data/bikes.json'
response = requests.get(url)
bikes = json.loads(response.text)

#### Inspect the Bikes JSON

Let's inspect the content of the JSON array in table form using the Pandas framework:

In [2]:
%pip -q install pandas
import pandas as pd

pd.DataFrame(bikes)

Note: you may need to restart the kernel to use updated packages.


,model,brand,price,type,specs,description
0,Jigger,Velorim,270,Kids bikes,"{'material': 'aluminium', 'weight': '10'}","Small and powerful, the Jigger is the best rid..."
1,Hillcraft,Bicyk,1200,Kids Mountain Bikes,"{'material': 'carbon', 'weight': '11'}",Kids want to ride with as little weight as pos...
2,Chook air 5,Nord,815,Kids Mountain Bikes,"{'material': 'alloy', 'weight': '9.1'}",The Chook Air 5 gives kids aged six years and...
3,Eva 291,Eva,3400,Mountain Bikes,"{'material': 'carbon', 'weight': '9.1'}","The sister company to Nord, Eva launched in 20..."
4,Kahuna,Noka Bikes,3200,Mountain Bikes,"{'material': 'alloy', 'weight': '9.8'}",Whether you want to try your hand at XC racing...
5,XBN 2.1 Alloy,Breakout,810,Road Bikes,"{'material': 'alloy', 'weight': '7.2'}",The XBN 2.1 Alloy is our entry-level road bike...
6,WattBike,ScramBikes,2300,eBikes,"{'material': 'alloy', 'weight': '15'}",The WattBike is the best e-bike for people who...
7,Soothe Electric bike,Peaknetic,1950,eBikes,"{'material': 'alloy', 'weight': '14.7'}","The Soothe is an everyday electric bike, from ..."
8,Secto,Peaknetic,430,Commuter bikes,"{'material': 'aluminium', 'weight': '10.0'}",If you struggle with stiff fingers or a kinked...
9,Summit,nHill,1200,Mountain Bike,"{'material': 'alloy', 'weight': '11.3'}",This budget mountain bike from nHill performs ...


Let's take a look at the structure of one of our bike JSON documents:

In [3]:
print(json.dumps(bikes[0], indent=2))

{
  "model": "Jigger",
  "brand": "Velorim",
  "price": 270,
  "type": "Kids bikes",
  "specs": {
    "material": "aluminium",
    "weight": "10"
  },
  "description": "Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids\u2019 pedal bike on the market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring to go. We say rare because this smokin\u2019 little bike is not ideal for a nervous first-time rider, but it\u2019s a true giddy up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one\u2019s need for speed. It\u2019s a single speed bike that makes learning to pump pedals simple and intuitive. It even has  a handle in the bottom of the saddle so you can easily help your child during training!  The Jigger is among the most lightweight children\u2019s bikes on the planet. It is designed so that 2-3 year-olds fit comfortably in a mo

#### Generating Text Embeddings using SentenceTransformers

We will use the SentenceTransformers framework to generate embeddings for the bikes descriptions. Sentence-BERT (SBERT) is a BERT model modification that produces consistent and contextually rich sentence embeddings. SBERT improves tasks like semantic search and text grouping by allowing for efficient and meaningful comparison of sentence-level semantic similarity.

Let's start by installing `sentence-transformers`:

In [4]:
%pip install -U -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.


### Selecting a suitable pre-trained Model

We must pick a suitable model based on the task when generating embeddings. In our case, we want to query for bicycles using short sentences against the longer bicycle descriptions. This is referred to as "Asymmetric Semantic Search," often employed in cases where the search query and the documents being searched are of different nature or structure. Suitable models for asymmetric semantic search include pre-trained MS MARCO Models. MS MARCO models are trained on the Microsoft MAchine Reading COmprehension (MS MARCO) dataset, optimizing them for understanding real-world queries and retrieving relevant responses. They are widely used in search engines, chatbots, and other AI applications. At the time of this writing, the highest performing MS MARCO model tuned for cosine-similarity from SentenceTranformers is `msmarco-distilbert-base-v4`. 

Let's load the model using the `SentenceTranformer` function:

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('msmarco-distilbert-base-v4')

Let's grab the description from the first bike in the JSON array:

In [6]:
from textwrap import TextWrapper

sample_description = bikes[0]['description']
TextWrapper(width=120).wrap(sample_description)

['Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids’ pedal bike on the',
 'market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring',
 'to go. We say rare because this smokin’ little bike is not ideal for a nervous first-time rider, but it’s a true giddy',
 'up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one’s need for',
 'speed. It’s a single speed bike that makes learning to pump pedals simple and intuitive. It even has  a handle in the',
 'bottom of the saddle so you can easily help your child during training!  The Jigger is among the most lightweight',
 'children’s bikes on the planet. It is designed so that 2-3 year-olds fit comfortably in a molded ride position that',
 'allows for efficient riding, balanced handling and agility. The Jigger’s frame design and gears work together so your',
 'buddingbiker can stand 

To generate the vector embeddings, we use the `encode` function:

In [7]:
embedding = embedder.encode(sample_description)

Let's take a peek at the first 5 elements of the generated vector:

In [8]:
print(embedding.tolist()[:5])

[0.20076487958431244, -0.13000741600990295, 0.3081613779067993, 0.20627960562705994, -0.3692358136177063]


Let's also extract the length of the vector embeddings generated by the model, which we'll store in the constant `VECTOR_DIMENSION`, which we'll use later when working with Redis indices:

In [9]:
VECTOR_DIMENSION = len(embedding)
VECTOR_DIMENSION

768

The chosen model generates vector embeddings of length `768` regardless of the length of the input text.

## Storing our bikes in Redis

### Redis Stack Setup

Now that we know how to vectorize the bikes descriptions, it's time to start working with Redis. If you are working in a Google Colab environment, the code below will download, install and start the Redis Stack Server:

In [10]:
in_colab = 'google.colab' in str(get_ipython())
if in_colab:
  !curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
  !tar -xvf redis-stack-server.tar.gz
  !./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes

Otherwise, you can download and run Redis Stack Server with docker. [See instructions here](https://redis.io/docs/stack/get-started/install/docker/).

### Redis Python Client

To interact with Redis, we'll install the [`redis-py`](https://github.com/redis/redis-py) client library, which encapsulates the commands to work with OSS Redis as well as Redis Stack:

In [11]:
%pip install -q redis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


#### Create a `redis-py` client and test the server

We'll instantiate the Redis client, connecting to the localhost on Redis' default port `6379`. By default, Redis returns binary responses; to decode them, we'll pass the `decode_responses` parameter set to `True`:

In [10]:
import redis
client = redis.Redis(host = 'localhost', port=6379, decode_responses=True)

Let's use Redis' [`PING`](https://redis.io/commands/ping/) command to check that Redis is up and running:

In [11]:
client.ping()

True

### Storing the Bikes as JSON Documents in Redis

Redis Stack includes [JSON](https://redis.io/docs/stack/json/) functionality. Like any other Redis data type, the JSON datatype allows you to use Redis commands to save, update, and retrieve JSON values. 
Since we already have the bikes data loaded in memory as the `bikes` JSON array. We will iterate over `bikes`, generate a suitable Redis key and store them in Redis using the [`JSON.SET`](https://redis.io/commands/json.set/) command. We'll do this [pipeline](https://redis.io/docs/manual/pipelining/) mode to minimize the round-trip times:

In [12]:
pipeline = client.pipeline()

for i, bike in enumerate(bikes, start=1):
    redis_key = f'bikes:{i:03}'
    pipeline.json().set(redis_key, '$', bike)

pipeline.execute()

[True, True, True, True, True, True, True, True, True, True, True]

Let's retrieve a specific value from one of the JSON bikes in Redis using a [JSONPath](https://goessner.net/articles/JsonPath/) expression:

In [13]:
client.json().get('bikes:010', '$.model')

['Summit']

### Vectorize all of the Bikes Descriptions

To vectorize all the descriptions in the database, we will first collect all the Redis keys for the bikes:

In [14]:
keys = sorted(client.keys('bikes:*'))
print(keys)

['bikes:001', 'bikes:002', 'bikes:003', 'bikes:004', 'bikes:005', 'bikes:006', 'bikes:007', 'bikes:008', 'bikes:009', 'bikes:010', 'bikes:011']


We'll use the keys as a parameter to the [`JSON.MGET`](https://redis.io/commands/json.mget/) command, along with the JSONPath expression `$.description` to collect the descriptions in a list which we will then pass to the `encode` method to get a list of vectorized embeddings:

In [15]:
import numpy as np

descriptions = client.json().mget(keys, '$.description')
descriptions = [item for sublist in descriptions for item in sublist]
embeddings = embedder.encode(descriptions).astype(np.float32).tolist()

Now we can add the vectorized descriptions to the JSON documents in Redis using the `JSON.SET` command to insert a new field in each of the documents under the JSONPath `$.description_embeddings`, once again we'll do this in pipeline mode:

In [16]:
pipeline = client.pipeline()

for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, '$.description_embeddings', embedding)

pipeline.execute()

[True, True, True, True, True, True, True, True, True, True, True]

Let's inspect one of the vectorized bike documents using the `JSON.GET` command:

In [17]:
print(json.dumps(client.json().get('bikes:010'), indent=2)) 

{
  "model": "Summit",
  "brand": "nHill",
  "price": 1200,
  "type": "Mountain Bike",
  "specs": {
    "material": "alloy",
    "weight": "11.3"
  },
  "description": "This budget mountain bike from nHill performs well both on bike paths and on the trail. The fork with 100mm of travel absorbs rough terrain. Fat Kenda Booster tires give you grip in corners and on wet trails. The Shimano Tourney drivetrain offered enough gears for finding a comfortable pace to ride uphill, and the Tektro hydraulic disc brakes break smoothly. Whether you want an affordable bike that you can take to work, but also take trail riding on the weekends or you\u2019re just after a stable, comfortable ride for the bike path, the Summit gives a good value for money.",
  "description_embeddings": [
    -0.5381147265434265,
    -0.494658887386322,
    -0.025176331400871277,
    0.6540350317955017,
    -0.06241431087255478,
    -0.6898810267448425,
    -0.5430219769477844,
    -0.590349555015564,
    0.5061323642730

When storing a vector embedding as part of a JSON datatype, the embedding is stored as a JSON array, in our case, under the field `description_embeddings` as shown.

### Making the bikes collection searchable

Redis Stack provides a powerful search engine ([RediSearch](https://redis.io/docs/stack/search/)) that introduces [commands](https://redis.io/docs/stack/search/commands/) to create and maintain search indices for both collections of HASHES and [JSON](https://redis.io/docs/stack/search/indexing_json/) documents.

To create a search index for the bikes collection, we'll use the [`FT.CREATE`](https://redis.io/commands/ft.create/) command:

```
1️⃣ FT.CREATE idx:bikes_vss ON JSON 
2️⃣  PREFIX 1 bikes: SCORE 1.0 
3️⃣  SCHEMA 
4️⃣    $.model TEXT WEIGHT 1.0 NOSTEM 
5️⃣    $.brand TEXT WEIGHT 1.0 NOSTEM 
6️⃣    $.price NUMERIC 
7️⃣    $.type TAG SEPARATOR "," 
8️⃣    $.description AS description TEXT WEIGHT 1.0 
9️⃣    $.description_embeddings AS vector VECTOR FLAT 6 TYPE FLOAT32 DIM 768 DISTANCE_METRIC COSINE
```

There is a lot to unpack here; let's take it from the top:

- 1️⃣ We start by specifying the name of the index; `idx:bikes` indexing keys of type `JSON`.
- 2️⃣ The keys being indexed are found using the `bikes:` key prefix.
- 3️⃣ The `SCHEMA` keyword marks the beginning of the schema field definitions.
- 4️⃣ Declares that field in the JSON document at the JSONPath `$.model` will be indexed as a `TEXT` field, allowing full-text search queries (disabling stemming).
- 5️⃣ The `$.brand` field will also be treated as a `TEXT` schema field.
- 6️⃣ The `$.price` field will be indexed as a `NUMERIC` allowing numeric range queries.
- 7️⃣ The `$.type` field will be indexed as a `TAG` field. Tag fields allow exact-match queries, and are suitable for categorical values.
- 8️⃣ The `$.description` field will also be indexed as a `TEXT` field.
- 9️⃣ Finally, the vector embeddings in `$.description_embeddings` are indexed as a `VECTOR` field and assigned to the alias `vector`. 
  
Let's break down the `VECTOR` schema field definition to better understand the inner workings of Vector Similarity in Redis:

* `FLAT`: Specifies the indexing method, which can be `FLAT` or `HNSW`. FLAT (brute-force indexing) provides exact results but at a higher computational cost, while HNSW (Hierarchical Navigable Small World) is a more efficient method that provides approximate results with lower computational overhead.
* `TYPE`: Set to `FLOAT32`. Current supported types are `FLOAT32` and `FLOAT64`.
* `DIM`: The length or dimension of our embeddings, which we determined previosly to be `768`.
* `DISTANCE_METRIC`: One of `L2`, `IP`, `COSINE`. 
  - `L2` stands for "Euclidean distance"; a straight-line distance between the vectors. Preferred when the absolute differences, including magnitude, matter most.
  - `IP` stands for "Inner Product"; IP measures the projection of one vector onto another. It emphasizes the angle between vectors rather than their absolute positions in the vector space.
  - `COSINE` stands for "Cosine Similarity"; a normalized form of inner product. This metric measures only the angle between two vectors, making it magnitude-independent.  
  - For our querying purposes, the direction of the vectors carry more meaning (indicating semantic similarity), and the magnitude is largely influenced by the length of the documents, therefore `COSINE` similarity is chosen. Also, our chosen embedding model is fine-tuned for Cosine Similarity.

The Python code below creates the Redis Search Index for the bikes collection equivalent to the previous raw `FT.CREATE` command:

In [ ]:
from redis.commands.search.field import TagField, TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

INDEX_NAME = 'idx:bikes_vss'
DOC_PREFIX = 'bikes:'

try:
    # check to see if index exists
    client.ft(INDEX_NAME).info()
    print('Index already exists!')
except:
    # schema
    schema = (
        TextField('$.model', no_stem=True, as_name='model'),
        TextField('$.brand', no_stem=True, as_name='brand'),
        NumericField('$.price', as_name='price'),
        TagField('$.type', as_name='type'),
        TextField('$.description', as_name='description'),
        VectorField('$.description_embeddings',
            'FLAT', {
                'TYPE': 'FLOAT32',
                'DIM': VECTOR_DIMENSION,
                'DISTANCE_METRIC': 'COSINE',
            },  as_name='vector'
        ),
    )

    # index Definition
    definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.JSON)

    # create Index
    client.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

#### Check the state of the index

After the `FT.CREATE` creates the index, the indexing process is automatically started in the background. In the blink of an eye, our 11 JSON documents should be indexed and ready to be searched. To corroborate that, we use the [`FT.INFO`](https://redis.io/commands/ft.info/) command to check some information and statistics on the index. Of particular interest are the number of documents successfully indexed, and the number of failures:  

In [22]:
info = client.ft(INDEX_NAME).info()

num_docs = info['num_docs']
indexing_failures = info['hash_indexing_failures']
total_indexing_time = info['total_indexing_time']
percent_indexed = int(info['percent_indexed']) * 100


print(f"{num_docs} documents ({percent_indexed} percent) indexed with {indexing_failures} failures in {float(total_indexing_time):.2f} milliseconds")

11 documents (100 percent) indexed with 0 failures in 10.02 milliseconds


### Structured Data Searches with Redis

The index `idx:bikes_vss` indexes the structured fields of our JSON documents `model`, `brand`, `price`, and `type`. It also indexes the unstructured free-form text `description` and the generated embeddings in `description_embeddings`. Before we dive deeper into Vector Similarity Search, we need to understand the basics of querying a Redis index. The Redis command of interest is [`FT.SEARCH`](https://redis.io/commands/ft.search/). Like a SQL select statement, an `FT.SEARCH` invocation can be as simple or as complex as needed. 

Let's try simple queries that give enough context to complete our VSS examples. For example, to retrieve all bikes where the `brand` is `Peaknetic`, we can use the following command:

```
FT.SEARCH idx:bikes_vss '@brand:Peaknetic'
```

The command will return all matching documents. With the inclusion of the vector embeddings, that's a little too verbose. If we wanted to only return specific fields from our JSON documents, for example, the document `id`, the `brand`, `model` and `price`, we could use:

```
FT.SEARCH idx:bikes_vss '@brand:Peaknetic' RETURN 4 id, brand, model, price
```

In the query, we are searching against a schema field of type `TEXT`. The equivalent Python code is:

In [23]:
query = (
    Query('@brand:Peaknetic').return_fields('id', 'brand', 'model', 'price')
)
client.ft(INDEX_NAME).search(query).docs

[Document {'id': 'bikes:009', 'payload': None, 'brand': 'Peaknetic', 'model': 'Secto', 'price': '430'},
 Document {'id': 'bikes:008', 'payload': None, 'brand': 'Peaknetic', 'model': 'Soothe Electric bike', 'price': '1950'}]

Let's say we only wanted bikes under `$1000`. We can add a numeric range clause to our query since the `price` field is indexed as `NUMERIC`:

In [24]:
query = (
    Query('@brand:Peaknetic @price:[0 1000]').return_fields('id', 'brand', 'model', 'price')
)
client.ft(INDEX_NAME).search(query).docs

[Document {'id': 'bikes:009', 'payload': None, 'brand': 'Peaknetic', 'model': 'Secto', 'price': '430'}]

### Semantic Searching with Vector Similarity Search

Now that the bikes collection is stored and properly indexed in Redis, we want to query them using short query prompts. Let's put our queries in a list so we can execute them in bulk:

In [25]:
queries = [
    'Bike for small kids',
    'Best Mountain bikes for kids',
    'Cheap Mountain bike for kids',
    'Female specific mountain bike',
    'Road bike for beginners',
    'Commuter bike for people over 60',
    'Comfortable commuter bike',
    'Good bike for college students',
    'Mountain bike for beginners',
    'Vintage bike',
    'Comfortable city bike'
]

We need to encode the query prompts to query the database using VSS. Just like we did with the descriptions of the bikes, we'll use the SentenceTransformers model to encode the queries:

In [26]:
encoded_queries = embedder.encode(queries)
len(encoded_queries)

11

#### Constructing a "Pure KNN" VSS Query

We'll start with a K-nearest neighbors (KNN) query. KNN is a foundational algorithm used in vector similarity search, where the goal is to find the most similar items to a given query item. Using the chosen distance metric, the KNN algorithm calculates the distance between the query vector and each vector in the database. It then returns the 'K' items with the smallest distances to the query vector. These are considered to be the most similar items.

The syntax for vector similarity KNN queries is `(*)=>[vector_similarity_query>]` where the `(*)` (the `*` meaning all) is the filter query for the search engine. That way, one can reduce the search space by filtering the collection on which the KNN algorithm operates. 

* The `$query_vector` represents the query parameter we'll use to pass the vectorized query prompt.
* The results will be filtered by `vector_score`, which is a field derived from the name of the field indexed as a Vector by appending `_score` to it, in our case, `vector` (the alias for `$.description_embeddings`). 
* Our query will return the `vector_score`, the `id` of the match documents, and the `$.brand`, `$.model`, and `$.description`. 
* Finally, to utilize a vector similarity query with the `FT.SEARCH` command, we must specify DIALECT 2 or greater.

In [27]:
query = (
    Query('(*)=>[KNN 3 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .return_fields('vector_score', 'id', 'brand', 'model', 'description')
     .dialect(2)
)

We pass the vectorized query as `$query_vector` to the search function to execute the query. The following code shows an example of creating a NumPy array from a vectorized query prompt (`encoded_query`) as a single precision float array and converting it into a compact, byte-level representation that we can pass as a Redis parameter:

```python .noeval
client.ft(INDEX_NAME).search(query, { 'query_vector': np.array(encoded_query, dtype=np.float32).tobytes() }).docs
````

With the template for the query in place, we can use a bit of Python code to execute all query prompts in a loop, passing the vectorized query prompts. We will then loop over the matched documents and create a list of results that will help up make a lovely Pandas table to visualize the results:

In [28]:
from IPython.display import display, HTML

def create_query_table(query, queries, encoded_queries, extra_params = {}):
    results_list = []
    for i, encoded_query in enumerate(encoded_queries):
        result_docs = client.ft(INDEX_NAME).search(query, { 'query_vector': np.array(encoded_query, dtype=np.float32).tobytes() } | extra_params).docs
        for doc in result_docs:
            vector_score = round(1 - float(doc.vector_score), 2)
            results_list.append({
                'query': queries[i], 
                'score': vector_score, 
                'id': doc.id,
                'brand': doc.brand,
                'model': doc.model,
                'description': doc.description
            })

    # Pretty-print the table
    queries_table = pd.DataFrame(results_list)
    queries_table.sort_values(by=['query', 'score'], ascending=[True, False], inplace=True)
    queries_table['query'] = queries_table.groupby('query')['query'].transform(lambda x: [x.iloc[0]] + ['']*(len(x)-1))
    queries_table['description'] = queries_table['description'].apply(lambda x: (x[:497] + '...') if len(x) > 500 else x)
    html = queries_table.to_html(index=False)
    display(HTML(html))

The query results show the individual queries' top 3 matches (our K parameter) along with the bike's id, brand, and model. The results show that "Female-specific mountain bike" is the query that resulted in the highest similarity score (`0.47`) for the 'Noka Bikes' brand 'Kahuna' bike model.

> "Whether you want to try your hand at XC racing or are looking for a lively trail bike that's just as inspiring on the climbs as it is over rougher ground, the Wilder is one heck of a bike built specifically for short women. Both the frames and components have been tweaked to include a women’s saddle, different bars and unique colourway."


From the bike description, we gather that this bike is an excellent all-purpose bike for women, and MS MARCO model-generated embeddings seem to have captured the semantics of the description.

In [29]:
create_query_table(query, queries, encoded_queries)

query,score,id,brand,model,description
Best Mountain bikes for kids,0.54,bikes:003,Nord,Chook air 5,"The Chook Air 5 gives kids aged six years and older a durable and uberlight mountain bike for their first experience on tracks and easy cruising through forests and fields. The lower top tube makes it easy to mount and dismount in any situation, giving your kids greater safety on the trails. The Chook Air 5 is the perfect intro to mountain biking."
,0.51,bikes:010,nHill,Summit,"This budget mountain bike from nHill performs well both on bike paths and on the trail. The fork with 100mm of travel absorbs rough terrain. Fat Kenda Booster tires give you grip in corners and on wet trails. The Shimano Tourney drivetrain offered enough gears for finding a comfortable pace to ride uphill, and the Tektro hydraulic disc brakes break smoothly. Whether you want an affordable bike that you can take to work, but also take trail riding on the weekends or you’re just after a stable,..."
,0.46,bikes:001,Velorim,Jigger,"Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids’ pedal bike on the market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring to go. We say rare because this smokin’ little bike is not ideal for a nervous first-time rider, but it’s a true giddy up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one’s need for speed. It’s a single..."
Bike for small kids,0.52,bikes:001,Velorim,Jigger,"Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids’ pedal bike on the market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring to go. We say rare because this smokin’ little bike is not ideal for a nervous first-time rider, but it’s a true giddy up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one’s need for speed. It’s a single..."
,0.45,bikes:007,ScramBikes,WattBike,"The WattBike is the best e-bike for people who still feel young at heart. It has a Bafang 500 watt geared hub motor that can reach 20 miles per hour on both steep inclines and city streets. The lithium-ion battery, which gets nearly 40 miles per charge, has a lightweight form factor, making it easier for seniors to use. It comes fully assembled (no convoluted instructions!) and includes a sturdy helmet at no cost. The Plush saddle softens over time with use. The included Seatpost, however, i..."
,0.41,bikes:003,Nord,Chook air 5,"The Chook Air 5 gives kids aged six years and older a durable and uberlight mountain bike for their first experience on tracks and easy cruising through forests and fields. The lower top tube makes it easy to mount and dismount in any situation, giving your kids greater safety on the trails. The Chook Air 5 is the perfect intro to mountain biking."
Cheap Mountain bike for kids,0.49,bikes:003,Nord,Chook air 5,"The Chook Air 5 gives kids aged six years and older a durable and uberlight mountain bike for their first experience on tracks and easy cruising through forests and fields. The lower top tube makes it easy to mount and dismount in any situation, giving your kids greater safety on the trails. The Chook Air 5 is the perfect intro to mountain biking."
,0.45,bikes:010,nHill,Summit,"This budget mountain bike from nHill performs well both on bike paths and on the trail. The fork with 100mm of travel absorbs rough terrain. Fat Kenda Booster tires give you grip in corners and on wet trails. The Shimano Tourney drivetrain offered enough gears for finding a comfortable pace to ride uphill, and the Tektro hydraulic disc brakes break smoothly. Whether you want an affordable bike that you can take to work, but also take trail riding on the weekends or you’re just after a stable,..."
,0.39,bikes:001,Velorim,Jigger,"Small and powerful, the Jigge

#### Hybrid Queries

"Pure KNN" queries, as in the previous section, evaluate a query against the whole space of vectors in a data collection. The larger the collection, the more computationally expensive the nearest neighbors' search will be, but in the real world, unstructured data does not live in isolation, and users expecting rich search experiences need to be able to search via a combination of structured and unstructured data. 

For example, users might arrive at your search interface with a brand preference in mind for the bikes dataset. Redis VSS queries can use this information to pre-filter the search space using a "primary filter query". In the following query definition, we pre-filter using the `brand` to consider only `Peaknetic` brand bikes. Notice that, before our primary filter query was `(*)`, AKA everything, but now we can narrow the search space using `(@brand:Peaknetic)` before the KNN query:

In [30]:
hybrid_query = (
    Query('(@brand:Peaknetic)=>[KNN 3 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .return_fields('vector_score', 'id', 'brand', 'model', 'description')
     .dialect(2)
)

Filtering by the `Peaknetic` brand, for which there are 2 bikes in our collection, we can see the results returned for each of the query prompts. The query with the highest returned similarity score is "Comfortable commuter bike", followed by "Road bike for beginners". By filtering by brand, we fulfill the users' preferences and reduce the KNN search space by %80.

In [31]:
create_query_table(hybrid_query, queries, encoded_queries)

query,score,id,brand,model,description
Best Mountain bikes for kids,0.30,bikes:008,Peaknetic,Soothe Electric bike,"The Soothe is an everyday electric bike, from the makers of Exercycle bikes, that conveys style while you get around the city. The Soothe lives up to its name by keeping your posture upright and relaxed for the ride ahead, keeping those aches and pains from riding at bay. It includes a low-step frame , our memory foam seat, bump-resistant shocks and conveniently placed thumb throttle."
,0.23,bikes:009,Peaknetic,Secto,"If you struggle with stiff fingers or a kinked neck or back after a few minutes on the road, this lightweight, aluminum bike alleviates those issues and allows you to enjoy the ride. From the ergonomic grips to the lumbar-supporting seat position, the Roll Low-Entry offers incredible comfort. The rear-inclined seat tube facilitates stability by allowing you to put a foot on the ground to balance at a stop, and the low step-over frame makes it accessible for all ability and mobility levels. Th..."
Bike for small kids,0.37,bikes:008,Peaknetic,Soothe Electric bike,"The Soothe is an everyday electric bike, from the makers of Exercycle bikes, that conveys style while you get around the city. The Soothe lives up to its name by keeping your posture upright and relaxed for the ride ahead, keeping those aches and pains from riding at bay. It includes a low-step frame , our memory foam seat, bump-resistant shocks and conveniently placed thumb throttle."
,0.25,bikes:009,Peaknetic,Secto,"If you struggle with stiff fingers or a kinked neck or back after a few minutes on the road, this lightweight, aluminum bike alleviates those issues and allows you to enjoy the ride. From the ergonomic grips to the lumbar-supporting seat position, the Roll Low-Entry offers incredible comfort. The rear-inclined seat tube facilitates stability by allowing you to put a foot on the ground to balance at a stop, and the low step-over frame makes it accessible for all ability and mobility levels. Th..."
Cheap Mountain bike for kids,0.21,bikes:008,Peaknetic,Soothe Electric bike,"The Soothe is an everyday electric bike, from the makers of Exercycle bikes, that conveys style while you get around the city. The Soothe lives up to its name by keeping your posture upright and relaxed for the ride ahead, keeping those aches and pains from riding at bay. It includes a low-step frame , our memory foam seat, bump-resistant shocks and conveniently placed thumb throttle."
,0.12,bikes:009,Peaknetic,Secto,"If you struggle with stiff fingers or a kinked neck or back after a few minutes on the road, this lightweight, aluminum bike alleviates those issues and allows you to enjoy the ride. From the ergonomic grips to the lumbar-supporting seat position, the Roll Low-Entry offers incredible comfort. The rear-inclined seat tube facilitates stability by allowing you to put a foot on the ground to balance at a stop, and the low step-over frame makes it accessible for all ability and mobility levels. Th..."
Comfortable city bike,0.42,bikes:008,Peaknetic,Soothe Electric bike,"The Soothe is an everyday electric bike, from the makers of Exercycle bikes, that conveys style while you get around the city. The Soothe lives up to its name by keeping your posture upright and relaxed for the ride ahead, keeping those aches and pains from riding at bay. It includes a low-step frame , our memory foam seat, bump-resistant shocks and conveniently placed thumb throttle."
,0.36,bikes:009,Peaknetic,Secto,"If you struggle with stiff fingers or a kinked neck or back after a few minutes on the road, this lightweight, aluminum bike alleviates those issues and allows you to enjoy the ride. From the ergonomic grips to the lumbar-supporting seat position, the Roll Low-Entry offers incredible comfort. The rear-inclined seat tube facilitates stability by allowing you to put a foot on the ground to balance at a stop, and the low step-over frame makes it accessible for all ability and m

#### Creating a VSS Range Query 

Range queries in Vector Similarity Search (VSS) involve retrieving items within a specific distance from a query vector. In this case, we consider "distance" to be the measure of similarity we've used to build our search indices; the smaller the distance, the more similar the items.

Let's say you want to find the bikes whose descriptions are within a certain distance from a query vector. We can use a range query to achieve this.
For example, the query command to return the top 4 documents within a 0.55 radius of a vectorized query would be as follows: 

```
1️⃣ FT.SEARCH idx:bikes_vss 
2️⃣   @vector:[VECTOR_RANGE $range $query_vector]=>{$YIELD_DISTANCE_AS: vector_score} 
3️⃣   SORTBY vector_score ASC
4️⃣   LIMIT 0 4 
5️⃣   DIALECT 2 
6️⃣   PARAMS 4 range 0.55 query_vector "\x9d|\x99>bV#\xbfm\x86\x8a\xbd\xa7~$?*...."
```

Where:

- 1️⃣ We use the `FT.SEARCH` command with our `idx:bikes_vss`.
- 2️⃣ and filter by the `vector` using the `VECTOR_RANGE` operator pasing the `$range` parameter, yield the vector distance between the vector field and the query result in a field named `vector_score`.
- 3️⃣ We sort the results by the yielded `vector_score`.
- 4️⃣ Limit the results to at most 4.
- 5️⃣ Once again, we set the RediSearch dialect to `2` to enable VSS functionality.
- 6️⃣ Finally we set the parameter values, `range` (`$range`) to `0.55` and the `query_vector` (`$query_vector`) to the encoded vectorized query. 

The equivalent Python query definition is shown below:

In [32]:
range_query = (
    Query('@vector:[VECTOR_RANGE $range $query_vector]=>{$YIELD_DISTANCE_AS: vector_score}') 
    .sort_by('vector_score')
    .return_fields('vector_score', 'id', 'brand', 'model', 'description')
    .paging(0, 4)
    .dialect(2)
)

Let's run the first query prompt in our collection of queries, "Bike for small kids", using the VSS range query defined (`range_query`). We can use the `create_query_table` utility function to execute the query passing the extra parameter `$range` as a dictionary:

In [35]:
create_query_table(range_query, queries[:1], encoded_queries[:1], {'range': 0.55})

query,score,id,brand,model,description
Bike for small kids,0.52,bikes:001,Velorim,Jigger,"Small and powerful, the Jigger is the best ride for the smallest of tikes! This is the tiniest kids’ pedal bike on the market available without a coaster brake, the Jigger is the vehicle of choice for the rare tenacious little rider raring to go. We say rare because this smokin’ little bike is not ideal for a nervous first-time rider, but it’s a true giddy up for a true speedster. The Jigger is a 12 inch lightweight kids bicycle and it will meet your little one’s need for speed. It’s a single..."
,0.45,bikes:007,ScramBikes,WattBike,"The WattBike is the best e-bike for people who still feel young at heart. It has a Bafang 500 watt geared hub motor that can reach 20 miles per hour on both steep inclines and city streets. The lithium-ion battery, which gets nearly 40 miles per charge, has a lightweight form factor, making it easier for seniors to use. It comes fully assembled (no convoluted instructions!) and includes a sturdy helmet at no cost. The Plush saddle softens over time with use. The included Seatpost, however, i..."


The query returns two bikes in the specified range of our vectorized query, both with scores at or below `0.55`.

## Wrapping Up

In this guide, we learned how Redis, using the Redis Stack distribution, provides powerful search capabilities over structured and unstructured data. Redis support for vector data can enrich your users' search experiences. 

The references below can help you learn more about Redis search capabilities:
* https://redis.io/docs/stack/search/
* https://redis.io/docs/stack/search/indexing_json/